# Download and Set-Up Paths

In [2]:
from __future__ import print_function
from IPython import display
import tensorflow.compat.v1 as tf
import sys
import os

# Download source code.
if "efficientnet_model.py" not in os.getcwd() or "imagenet.py" not in os.getcwd():
  !git clone --depth 1 https://github.com/rezabasiri/EfficientNetDFU.git
  display.clear_output()
  os.chdir('EfficientNetDFU')
  sys.path.append('.')
  os.chdir('common')
  sys.path.append('.')
  os.chdir('..')
else:
  !git pull

In [3]:
MODEL = 'efficientnet-b0'

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{m}.tar.gz
    !tar xf {m}.tar.gz
    display.clear_output()
    !rm -rf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model checkpoint in {}'.format(ckpt_path))

Use model checkpoint in /content/EfficientNetDFU/efficientnet-b0



# Set-Up Data


In [ ]:
# Prepare Training Data
!rm -rf *.zip *.tar tfrecord/ val2017/
!mkdir tfrecord
!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_coco_tfrecord.py \
      --image_dir="/cluster/home/t62003uhn/Dataset/Chess/Images" \
      --object_annotations_file="/cluster/home/t62003uhn/Dataset/Chess/annotations/instances_Images.json" \
      --output_file_prefix=tfrecord/train \
      --num_shards=1

2020-07-28 02:56:50.421860: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
I0728 02:56:51.973358 140315061675904 create_coco_tfrecord.py:288] writing to output path: tfrecord/train


In [ ]:
# Prepare Validation Data
!PYTHONPATH=".:$PYTHONPATH"  python dataset/create_coco_tfrecord.py \
      --image_dir="/cluster/home/t62003uhn/Dataset/ChessVal/ImagesVal" \
      --object_annotations_file="/cluster/home/t62003uhn/Dataset/ChessVal/annotations/instances_ImagesVal.json" \
      --output_file_prefix=tfrecord/validation \
      --num_shards=1

In [ ]:
file_pattern = 'train-*-of-00001.tfrecord' # Update to match the number of shards for training set
file_patternVal = 'validation-*-of-00001.tfrecord' # Update to match the number of shards for valid set

images_per_epoch = 57 * len(tf.io.gfile.glob('tfrecord/' + file_pattern))
images_per_epoch = images_per_epoch // 8 * 8  # round to 64.

images_per_epochVal = 57 * len(tf.io.gfile.glob('tfrecord/' + file_patternVal))
images_per_epochVal = images_per_epochVal // 8 * 8  # round to 64.

print('images_per_epoch = {}'.format(images_per_epoch))
print('images_per_epochVal = {}'.format(images_per_epochVal))

# Create the Model and Run Train-Eval

In [ ]:
!mkdir model_dir/
!python main.py --mode=train_and_eval \
    --data_dir=tfrecord \
    --model_name={MODEL} \
    --model_dir=model_dir/{MODEL}-finetune \
    --train_batch_size=8 \
    --eval_batch_size=8 \
    --num_eval_images=399 \
    --num_train_images=1601 \
    --num_label_classes=1 \
    --use_tpu=false \
    --train_steps=50000 \
    --steps_per_eval=500 \
    --moving_average_decay=0 \
    --base_learning_rate=0.01 \

In [ ]:
#{images_per_epochVal}  
#{images_per_epoch} 
# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt -O labels_map.txt

In [ ]:
# import eval_ckpt_main as eval_ckpt
# import tensorflow.compat.v1 as tf

# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{model_name}.tar.gz -O {model_name}.tar.gz
# !tar xf {model_name}.tar.gz
# ckpt_dir = model_name
# !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt -O labels_map.txt
# labels_map_file = 'labels_map.txt'


# image_files = [image_file]
# eval_driver = eval_ckpt.get_eval_driver(model_name)
# pred_idx, pred_prob = eval_driver.eval_example_images(
#     ckpt_dir, image_files, labels_map_file)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir model_dir/